<a href="https://colab.research.google.com/github/gardnmi/Notebooks/blob/master/autoguan_college_football_picks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autogluon
!pip install mxnet

In [ ]:
!pip install -U ipykernel

In [3]:
import autogluon as ag
from autogluon import TabularPrediction as task

In [81]:
drop_columns = ['spread_result', 'pre_game_spread' , 'pre_game_home_win_prob', 'pre_game_away_win_prob']

train_data = task.Dataset(file_path='train.csv')
train_data = train_data.drop(drop_columns, axis=1) # use for regression
# train_data = train_data.drop('spread_target', axis=1) # use for classifier

Loaded data from: train.csv | Columns = 434 / 434 | Rows = 4729 -> 4729


In [82]:
# label_column = 'spread_result' # use for classifier
label_column = 'spread_target' # use for regression

In [60]:
predictor = task.fit(train_data=train_data, label=label_column, problem_type = 'regression')

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200919_045034/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200919_045034/
AutoGluon Version:  0.0.13
Train Data Rows:    4729
Train Data Columns: 430
Preprocessing data ...
Feature Generator processed 4729 data points with 429 features
Original Features (raw dtypes):
	int64 features: 4
	object features: 7
	bool features: 2
	float64 features: 416
Original Features (inferred dtypes):
	int features: 4
	object features: 6
	datetime features: 1
	bool features: 2
	float features: 416
Generated Features (special dtypes):
	datetime features: 1
Processed Features (raw dtypes):
	int features: 5
	bool features: 2
	float features: 416
	category features: 6
Processed Features:
	datetime features: 1
	int features: 4
	bool features: 2
	float features: 416
	category features: 6
	Data preprocessing and feature engineering runtime = 1.07s ...
AutoGluon will gauge predictive performance us

[1000]	train_set's rmse: 0.00112501	valid_set's rmse: 16.9777


	-16.9776	 = Validation root_mean_squared_error score
	657.87s	 = Training runtime
	0.38s	 = Validation runtime
Fitting model: weighted_ensemble_k0_l1 ...
	-16.5285	 = Validation root_mean_squared_error score
	0.55s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1091.79s ...


In [83]:
drop_columns = ['spread_result', 'pre_game_spread' , 'pre_game_home_win_prob', 'pre_game_away_win_prob']

test_data = task.Dataset(file_path='test.csv')
y_test = test_data[label_column] 
test_data = test_data.drop(drop_columns, axis=1) # use for regression

Loaded data from: test.csv | Columns = 434 / 434 | Rows = 879 -> 879


In [84]:
y_train_pred = predictor.predict(train_data)
y_test_pred = predictor.predict(test_data)

perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_test_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: 17.970572602799994
Evaluations on test data:
{
    "root_mean_squared_error": 17.970572602799994,
    "mean_absolute_error": 14.22694264626103,
    "explained_variance_score": 0.3882426486045142,
    "r2_score": 0.3861712472265425,
    "pearson_correlation": 0.6252474772398675,
    "mean_squared_error": 322.9414796725056,
    "median_absolute_error": 11.1981201171875
}


In [87]:
drop_columns = ['spread_target', 'pre_game_spread' , 'pre_game_home_win_prob', 'pre_game_away_win_prob']

train_data = task.Dataset(file_path='train.csv')
train_data = train_data.drop(drop_columns, axis=1) # use for classifier
train_data = pd.concat([train_data, pd.Series(y_train_pred, name='spread_pred')], axis=1)

Loaded data from: train.csv | Columns = 434 / 434 | Rows = 4729 -> 4729


In [88]:
label_column = 'spread_result' # use for classifier

In [89]:
predictor = task.fit(train_data=train_data, label=label_column, problem_type = 'binary')

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200919_051800/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200919_051800/
AutoGluon Version:  0.0.13
Train Data Rows:    4729
Train Data Columns: 431
Preprocessing data ...
Selected class <--> label mapping:  class 1 = Away, class 0 = Home
Train Data Class Count: 2
Feature Generator processed 4729 data points with 430 features
Original Features (raw dtypes):
	int64 features: 4
	object features: 7
	bool features: 2
	float64 features: 416
	float32 features: 1
Original Features (inferred dtypes):
	int features: 4
	object features: 6
	datetime features: 1
	bool features: 2
	float features: 417
Generated Features (special dtypes):
	datetime features: 1
Processed Features (raw dtypes):
	int features: 5
	bool features: 2
	float features: 417
	category features: 6
Processed Features:
	datetime features: 1
	int features: 4
	bool features: 2
	float features: 417
	category features

In [90]:
drop_columns = ['spread_target', 'pre_game_spread' , 'pre_game_home_win_prob', 'pre_game_away_win_prob']

test_data = task.Dataset(file_path='test.csv')
y_test = test_data[label_column] 

test_data = test_data.drop(drop_columns, axis=1) # use for classifier
test_data = pd.concat([test_data, pd.Series(y_test_pred, name='spread_pred')], axis=1)

y_test_pred = predictor.predict(test_data)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_test_pred, auxiliary_metrics=True)

Loaded data from: test.csv | Columns = 434 / 434 | Rows = 879 -> 879
Evaluation: accuracy on test data: 0.49715585893060293
Evaluations on test data:
{
    "accuracy": 0.49715585893060293,
    "accuracy_score": 0.49715585893060293,
    "balanced_accuracy_score": 0.4994769224395094,
    "matthews_corrcoef": -0.0010774626957728971,
    "f1_score": 0.49715585893060293
}
Detailed (per-class) classification report:
{
    "Away": {
        "precision": 0.48990825688073397,
        "recall": 0.6194895591647331,
        "f1-score": 0.5471311475409836,
        "support": 431
    },
    "Home": {
        "precision": 0.5089820359281437,
        "recall": 0.3794642857142857,
        "f1-score": 0.43478260869565216,
        "support": 448
    },
    "accuracy": 0.49715585893060293,
    "macro avg": {
        "precision": 0.4994451464044388,
        "recall": 0.4994769224395094,
        "f1-score": 0.4909568781183179,
        "support": 879
    },
    "weighted avg": {
        "precision": 0.499629